In [278]:
import json
import re
import pickle
import sympy as smp
from types import SimpleNamespace

In [284]:

class MathJSON:
    def __init__(self, objectJson):
        self.objectJson = objectJson
        
    def __add__(self, other):
        return MathJSON({ 'Add' : [self.objectJson, other.objectJson] })
    
    def __mul__(self, other):
        return MathJSON({ 'Mul' : [self.objectJson, other.objectJson] })
    
    def __sub__(self, other):
        return MathJSON({ 'Sub' : [self.objectJson, other.objectJson] })
    
    def __truediv__(self, other):
        return MathJSON({ 'Div' : [self.objectJson, other.objectJson] })

    def __eq__(self, other):
        return MathJSON({ 'Eq' : [self.objectJson, other.objectJson] })

class TensorStringParser:
    def __init__(self, string_representation):
        self.string_representation = string_representation


In [312]:
# Imagine the user enters a string for the equation for the Christofel Symbols:

# He can either A: assign it with an equals sign or B: simply input the string

input_box = ' integrate(diff(G^{a=4}^{=0}, x), x)*(1/2)*G^{a=4}^{=0}*(D_{c}*G_{b}_{d} + D_{d}*G_{b}_{c} - D_{b}*G_{c}_{d}) '

# Steps:

tokens  = ['(','1','/','2',')','*','G^{a}^{b}','*','(','D_{c}','*','G_{b}_{d}',' +','D_{d}','*','G_{b}_{c}',' - ','D_{b}','*','G_{c}_{d}',')']

parsed_tokens = [
                '(',
                 MathJSON({'integer' : '1'}),
                 '/',
                 MathJSON({'integer' : '2'}),
                 ')',
                 '*',
                 MathJSON({'tensor_string_representation' : 'G^{a}^{b}'}),
                 '*',
                 '(',
                 MathJSON({'tensor_string_representation' : 'D_{c}'}),
                 '*',
                 MathJSON({'tensor_string_representation' : 'G_{b}_{d}'}),
                 '+',
                 MathJSON({'tensor_string_representation' : 'D_{d}'}),
                 '*',
                 MathJSON({'tensor_string_representation' : 'G_{b}_{c}'}),
                 '-',
                 MathJSON({'tensor_string_representation' : 'D_{b}'}),
                 '*',
                 MathJSON({'tensor_string_representation' : 'G_{c}_{d}'}),
                 ')'
                 ]

# It is at this point which if the user has entered a value which is in the store. In this case, the TensorStringParser will look for value in database 
# and use those values stored as its components.

"""
----- SHUTTING YARD ALGORITHM: ------

1.  While there are tokens to be read:
2.        Read a token
3.        If it's a MathJSON, add it to queue
4.        If it's an operator
5.               While there's an operator on the top of the stack with greater precedence:
6.                       Pop operators from the stack onto the output queue
7.               Push the current operator onto the stack
8.        If it's a left bracket push it onto the stack
9.        If it's a right bracket 
10.            While there's not a left bracket at the top of the stack:
11.                     Pop operators from the stack onto the output queue.
12.             Pop the left bracket from the stack and discard it
13. While there are operators on the stack, pop them to the queue
"""

# We now have a consistent list type -> operations and objects wrapped around the MathJSON class.
# We then perform an algorithm in this list to perform the calculations in the correct order.

MathJson = {'Mul': [{'Mul': [{'Div': [{'integer': '1'}, {'integer': '2'}]},
    {'tensor_string_representation': 'G^{a}^{b}'}]},
  {'Sub': [{'Add': [{'Mul': [{'tensor_string_representation': 'D_{c}'},
        {'tensor_string_representation': 'G_{b}_{d}'}]},
      {'Mul': [{'tensor_string_representation': 'D_{d}'},
        {'tensor_string_representation': 'G_{b}_{c}'}]}]},
    {'Mul': [{'tensor_string_representation': 'D_{b}'},
      {'tensor_string_representation': 'G_{c}_{d}'}]}]}]}

# Next we parse this into ComputeMathJson object which will, depending on the key value of the object, return the object we want:
# integers -> convert to integer -> return integer
# tensor_string_representation -> return TensorStringParser( tensor string )
# tensor_json_object -> return TensorJsonParser( json object )

# LAST STEP: Pass the MathJson object into ComputeMathJson object, which should return the answer of the expression AND/OR store 


In [297]:
class Tokenizer:

    def __init__(self, string_expression):
        self.string_expression = string_expression

    def match_tensors(self):
        string = self.string_expression
        pattern = lambda x : "([a-zA-Z]+)([_^]\{[a-zA-Z]+\}|[_^]\{[a-zA-Z]+\=[0-9]}){" + str(x) + "}(?=(\*|\)|\+|\-|\/|$))"
        Total = [[x for x in re.finditer(pattern(j), string)] for j in range(1, 11)]
        return [tensor.group() for nested in Total for tensor in nested]

    def match_operators(self):
        """
            Make checks as to what the name of the function is:
                - Integrate
                - solve
                - diff
                - subs
                or just declared functions
                - f(x)
                - etc...
            Then, make checks on the names and structure of the parameters.
            And finally parse the object into the relevant sympy object and return it into the MathJSON object.
        """
        Input = self.string_expression.replace(' ', '')
        Function = '(?<![a-zA-Z])' + '([a-zA-Z]+)' + '(\(([a-z]+\))' + '|' + '\(([a-z]+\,)*[a-z]\))'
        return [x for x in re.finditer(Function, Input)]

Tensors_list = Tokenizer("R_{a}_{s}+T_{a=4}+Ricci_{a=2}_{s=3}_{d}*f(t,f)+Ricci_{a=2}_{s=3}_{d}-I_{mu}^{g}_{f}_{s}").match_tensors()
Tensors_list


['T_{a=4}',
 'R_{a}_{s}',
 'Ricci_{a=2}_{s=3}_{d}',
 'Ricci_{a=2}_{s=3}_{d}',
 'I_{mu}^{g}_{f}_{s}']

In [359]:
class DataBase:

    def __init__(self):
        self.data = {}

database = DataBase()

class MathJSON:
    def __init__(self, objectJson):
        self.objectJson = objectJson
        
    def __add__(self, other):
        return MathJSON({ 'Add' : [self.objectJson, other.objectJson] })
    
    def __mul__(self, other):
        return MathJSON({ 'Mul' : [self.objectJson, other.objectJson] })
    
    def __sub__(self, other):
        return MathJSON({ 'Sub' : [self.objectJson, other.objectJson] })
    
    def __truediv__(self, other):
        return MathJSON({ 'Div' : [self.objectJson, other.objectJson] })

    def __eq__(self, other):
        return MathJSON({ 'Eq' : [self.objectJson, other.objectJson] })
    
class ComputeMathJson:
    def __init__(self, x):
        self.operations = {"Add" : lambda List : ComputeMathJson(List[0])+ComputeMathJson(List[1]),
                           "Mul" : lambda List : ComputeMathJson(List[0])*ComputeMathJson(List[1]),
                           "Sub" : lambda List : ComputeMathJson(List[0])-ComputeMathJson(List[1]),
                           "Div" : lambda List : ComputeMathJson(List[0])/ComputeMathJson(List[1]),
                           "Diff" : lambda List : smp.diff(List[0], List[1]),
                           "Int" : lambda List : smp.integrate(List[0], List[1]),
                           "Eq" : lambda List : database.data.update({List[0] : ComputeMathJson(List[1]).x}),
                           "integer" : lambda x : int(x),
                           "tensor" : lambda x : TensorStringParser(x)}
        
        if isinstance(x, dict):
            if [i for i in x][0] == "Diff":
                self.x = self.operations["Diff"](x["Diff"])
            elif [i for i in x][0] == "Int":
                self.x = self.operations["Int"](x["Int"])
            elif [i for i in x][0] == "integer":
                self.x = self.operations["integer"](x["integer"])
            else:
                self.x = self.operations[[i for i in x][0]](x[[i for i in x][0]])
        else:
            self.x = x
            
            
    def __add__(self, other):
        return self.x + other.x
    
    def __mul__(self, other):
        return self.x*other.x
    
    def __sub__(self, other):
        return self.x-other.x
    
    def __truediv__(self, other):
        return self.x/other.x
    

class MathJsonToString:
    def __init__(self, x):
        self.operations = {"Add" : lambda List : "(" + str(List[0]) + "+" + str(List[1]) + ")",
                           "Mul" : lambda List : "(" + str(List[0]) + "+" + str(List[1]) + ")",
                           "Sub" : lambda List : "(" + str(List[0]) + "+" + str(List[1]) + ")",
                           "Div" : lambda List : "(" + str(List[0]) + "+" + str(List[1]) + ")",
                           "Diff" : lambda List : "diff(" + str(List[0]) + "," + str(List[1]) + ")",
                           "Int" : lambda List : "integrate(" + str(List[0]) + "," + str(List[1]) + ")",
                           "Eq" : lambda List : str(List[0]) + " = " + str(List[1]),
                           "integer" : lambda x : str(x),
                           "tensor" : lambda x : x}
        
        if isinstance(x, dict):
            if [i for i in x][0] == "Diff":
                self.x = self.operations["Diff"](x["Diff"])
            elif [i for i in x][0] == "Int":
                self.x = self.operations["Int"](x["Int"])
            elif [i for i in x][0] == "integer":
                self.x = self.operations["integer"](x["integer"])
            else:
                self.x = self.operations[[i for i in x][0]](x[[i for i in x][0]])
        else:
            self.x = x

    def get_string(self):
        return self.x

class TensorTokenParser:
    
    """
    Class which converts:
    
    string tensor representation -> Tensor Object
    
    """
    
    def __init__(self, tensor_token):
        self.tensor_string = tensor_string
        
    
    
string = "Ricci_{a=0}_{s=1}+T_{a=4}+Ricci_{a=2}_{s=3}_{d}+Ricci_{a=2}_{s=3}_{d}-I_{mu}^{g}_{f}_{s}"
string1 = "Ricci(fbv,f) fr"

def Functions(Input):
    """
        Make checks as to what the name of the function is:
            - Integrate
            - solve
            - diff
            - subs
            or just declared functions
            - f(x)
            - etc...
        Then, make checks on the names and structure of the parameters.
        And finally parse the object into the relevant sympy object and return it into the MathJSON object.
    """
    Function = '(?<![a-zA-Z])' + '([a-zA-Z]+)' + '(\(([a-z]+\))' + '|' + '\(([a-z]+\,)*[a-z]\))'
    RemoveSpaces = Input.replace(' ', '')
    FunctionList = []
    return [x for x in re.finditer(Function, RemoveSpaces)]
    
def rank_tensor_match(string):
    pattern = lambda x : "([a-zA-Z]+)([_^]\{[a-zA-Z]+\}|[_^]\{[a-zA-Z]+\=[0-9]}){" + str(x) + "}(?=(\*|\)|\+|\-|\/|$))"
    Total = [[x for x in re.finditer(pattern(j), string)] for j in range(1, 11)]
    return [tensor.group() for nested in Total for tensor in nested]

rank_tensor_match(string)

['T_{a=4}',
 'Ricci_{a=0}_{s=1}',
 'Ricci_{a=2}_{s=3}_{d}',
 'Ricci_{a=2}_{s=3}_{d}',
 'I_{mu}^{g}_{f}_{s}']

In [354]:
re.search('(?<![a-zA-Z])' + '([a-zA-Z]+)' + '(\(([a-z]+\))' + '|' + '\(([a-z]+\,)*[a-z]\))', "dwdm+integrate(f,ff,f,f,f)")

<re.Match object; span=(5, 26), match='integrate(f,ff,f,f,f)'>

In [355]:
pattern2 = lambda x : "([a-zA-Z]+)([_^]\{[a-z]+\}|[_^]\{[a-z]+\=[0-9]}){" + str(x) + "}(?=(\*|\)|\+|\-|\/|$))"
pattern2(3)

'([a-zA-Z]+)([_^]\\{[a-z]+\\}|[_^]\\{[a-z]+\\=[0-9]}){3}(?=(\\*|\\)|\\+|\\-|\\/|$))'

In [361]:
f = MathJSON("New") == MathJSON(3)*(MathJSON({"Diff" : [smp.symbols('x')**2, smp.symbols('x')]})+MathJSON(4))+MathJSON(4)+MathJSON(7)
MathJsonToString(f.objectJson).get_string()

"New = {'Add': [{'Add': [{'Mul': [3, {'Add': [{'Diff': [x**2, x]}, 4]}]}, 4]}, 7]}"

In [317]:
ComputeMathJson(f.objectJson)
database.data['New']

6*x + 23

In [318]:
ComputeMathJson({'Div': [{'integer': '1'}, {'integer': '2'}]}).x

0.5

In [319]:
import ast
foo = "[['Cheese', 72], ['Milk', 45], ['Bread', 22]]"
ast.literal_eval(foo)
[['Cheese', 72], ['Milk', 45], ['Bread', 22]]

[['Cheese', 72], ['Milk', 45], ['Bread', 22]]

In [320]:
ast.parse("""5+3""")